## Homework 1: Search hyperparameters for a deep feedforward neural network
In this homework, you are going to 
1. practice using Keras to implement a neural network and search its hyperparameters for handwritten digits classification.
2. The framework, e.g., the main function and functions' names, input, and output, has been defined. You are required to complete the create_NN, nn_params_search, retrain_best_nn, performance_acc functions.
3. Add your code in the following blocks, and do not change other places.

```python

    ## add your code here
    
    ##
```

### Student information
    1. Your name: Steve Dodd
    2. Department: CS
    3. Graduate

### TA grading: XX/100
    1. 1.1?/10
    2. 1.2 ?/10
    3. create_NN: ?/30
    4. nn_params_search: ?/30
    5. retrain_best_nn: ?/10
    6: performance_acc: ?/10

In [20]:
# Homework 1: Solution
import keras
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix, accuracy_score
import numpy as np

In [21]:
# Load data and data standardization
def load_data():
    '''Load the MNIST dataset'''
    
    (X_train, y_train), (X_test, y_test) = keras.datasets.mnist.load_data()
    return (X_train, y_train, X_test, y_test)

def data_std(X_train, X_test):
    '''Data standardization
    
    Parameters
    ----------
    X_train: origianl training set
    X_test: original test set
    
    Returns
    -------
    X_train_std: rescaled training set
    X_test_std: rescaled test set
    '''
    sc = StandardScaler()
    sc.fit(X_train)
    X_train_std = sc.transform(X_train)
    X_test_std = sc.transform(X_test)
    
    return X_train_std, X_test_std

In [22]:
def create_NN(n_features = 784, n_outputs = 10): # 30 points
    '''create a deep feedforwad neural network using keras
    
    Parameters
    -----------
    n_features: the number of input features/units
    n_output: the number of output units
    
    Returns
    -------
    myNN: the neural network model
    
    '''
    ## add your code here
    myNN = keras.models.Sequential()
    myNN.add(keras.layers.Dense(32, activation='relu', input_dim=n_features))
    myNN.add(keras.layers.Dense(64, activation='relu'))
    myNN.add(keras.layers.Dense(64, activation='relu'))
    myNN.add(keras.layers.Dense(n_outputs, activation='softmax'))
    myNN.compile(optimizer='rmsprop',
              loss='categorical_crossentropy',
              metrics=['accuracy'])
    ##
    
    return myNN

In [23]:
def nn_params_search(nn, X_train, y_train): # 30 points
    '''Search best paramaters for svm classifier
    
    Parameters
    ----------
    X_train: features
    y_train: target of the input

    
    Returns
    -------
    best_params_
    
    Example grid: (you can customize the search graid by youself)
    param_grid = [{'batch_size': [64, 128], 'epochs' : [10, 30, 50]}]
        
    '''
    ## add your code here
    param_grid = [{'batch_size': [64, 128], 'epochs' : [10, 30, 50]}]
    gs_cv = GridSearchCV(estimator=nn, param_grid=param_grid, cv=5)
    gs_cv.fit(X_train, y_train)
    ##
    
    return gs_cv.best_params_

In [24]:
def retrain_best_nn(best_params, X_train, y_train): # 10 points
    '''
    Retrain a svm classifier using the best parameters
    
    Paramters
    ----------
    best_params:
    X_train: data input of the training set
    y_train: target of the input
    
    Returns
    ---------
    bestNN: the nn classifier trained using the best parameters
    
    '''
    ## add your code here  
    bestNN = create_NN()
    bestNN.fit(X_train,
               y_train,
               validation_split=0.2,
               verbose=1,
               batch_size = best_params['batch_size'],
               epochs = best_params['epochs'])
    ##
    
    return bestNN

In [25]:
def performance_acc(y, y_pred): # 10 points
    ''' calculate the concusion matrix and average accuracy
    
        Parameters
        ----------
        y: real target
        y_pred: prediction
        
        Returns
        -------
        cm: confusion matrix
        acc: accuracy
    '''
    ## add your code here
    from sklearn.metrics import accuracy_score
    cm = confusion_matrix(y, y_pred)
    acc = accuracy_score(y, y_pred)
    ##
    
    return cm, acc

In [26]:
from keras.wrappers.scikit_learn import KerasClassifier

if __name__ == '__main__':
    
    #Task 1. load the dataset
    (X_train, y_train, X_test, y_test) = load_data()
    
    # 1.1 reshape the training and test sets to N * 784. 10 points
    ## add your code here
    X_train_1 = X_train.reshape(X_train.shape[0], 784)
    X_test_1 = X_test.reshape(X_test.shape[0], 784)
    print('1. X_train: {}, X_train_1: {}'.format(X_train.shape, X_train_1.shape))
    ##
    
    # 1.2 transform y_train to one-hot vectors using keras.utils.to_categorical. 10 points
    ## add your code here
    num_classes = 10
    y_train_onehot = keras.utils.to_categorical(y_train, num_classes)
    print('y_train_onehot: {}'.format(y_train_onehot.shape))
    ##
    
    #Task 2. create a deep feedforward neural network
    myNN = create_NN(X_train_1.shape[1], y_train_onehot.shape[1])
    myNN.summary()
    myNN1 = KerasClassifier(build_fn = create_NN, batch_size = 64, epochs = 50)
    
    #Task 3. Search best paprameters, and report the performance
    best_params = nn_params_search(myNN1, X_train_1, y_train)
    print('Best parameters: ', best_params)
    
    bestNN = retrain_best_nn(best_params, X_train_1, y_train_onehot)
    y_test_pred = bestNN.predict_classes(X_test_1)
    cm, acc = performance_acc(y_test, y_test_pred)
    print('Confusion matrix:\n', cm)
    print('Accuracy =    {:.3f}%'.format(acc*100))
    
    #Task 4. Search best nn parameters after data standardization, and report the performance
    X_train_std, X_test_std = data_std(X_train_1, X_test_1)
    
    best_params =  nn_params_search(myNN1, X_train_std, y_train)
    print('Best parameters: ', best_params)
    
    bestNN_std = retrain_best_nn(best_params, X_train_std, y_train_onehot)
    y_test_std_pred = bestNN_std.predict_classes(X_test_std)
    cm1, acc1 = performance_acc(y_test, y_test_std_pred)
    print('Confusion matrix:\n', cm1)
    print('Accuracy =    {:.3f}%'.format(acc1*100))

1. X_train: (60000, 28, 28), X_train_1: (60000, 784)
y_train_onehot: (60000, 10)
Model: "sequential_69"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_269 (Dense)            (None, 32)                25120     
_________________________________________________________________
dense_270 (Dense)            (None, 64)                2112      
_________________________________________________________________
dense_271 (Dense)            (None, 64)                4160      
_________________________________________________________________
dense_272 (Dense)            (None, 10)                650       
Total params: 32,042
Trainable params: 32,042
Non-trainable params: 0
_________________________________________________________________
Epoch 1/10
48000/48000 [==============================] - 1s 15us/step - loss: 1.3535 - accuracy: 0.7619
Epoch 2/10
48000/48000 [==============================] - 1s 

48000/48000 [==============================] - 1s 13us/step - loss: 0.1798 - accuracy: 0.9618
Epoch 17/30
48000/48000 [==============================] - 1s 13us/step - loss: 0.1760 - accuracy: 0.9631
Epoch 18/30
48000/48000 [==============================] - 1s 13us/step - loss: 0.1740 - accuracy: 0.9647
Epoch 19/30
48000/48000 [==============================] - 1s 13us/step - loss: 0.1681 - accuracy: 0.9636
Epoch 20/30
48000/48000 [==============================] - 1s 13us/step - loss: 0.1723 - accuracy: 0.9644
Epoch 21/30
48000/48000 [==============================] - 1s 13us/step - loss: 0.1713 - accuracy: 0.9654
Epoch 22/30
48000/48000 [==============================] - 1s 13us/step - loss: 0.1735 - accuracy: 0.9638
Epoch 23/30
48000/48000 [==============================] - 1s 13us/step - loss: 0.1766 - accuracy: 0.9654
Epoch 24/30
48000/48000 [==============================] - 1s 13us/step - loss: 0.1672 - accuracy: 0.9647
Epoch 25/30
48000/48000 [==============================] -

48000/48000 [==============================] - 1s 22us/step - loss: 1.1007 - accuracy: 0.7668
Epoch 2/30
48000/48000 [==============================] - 1s 21us/step - loss: 0.3695 - accuracy: 0.9071
Epoch 3/30
48000/48000 [==============================] - 1s 18us/step - loss: 0.2847 - accuracy: 0.9273
Epoch 4/30
48000/48000 [==============================] - 1s 18us/step - loss: 0.2554 - accuracy: 0.9357
Epoch 5/30
48000/48000 [==============================] - 1s 18us/step - loss: 0.2280 - accuracy: 0.9417
Epoch 6/30
48000/48000 [==============================] - 1s 18us/step - loss: 0.2194 - accuracy: 0.9453
Epoch 7/30
48000/48000 [==============================] - 1s 17us/step - loss: 0.2075 - accuracy: 0.9496
Epoch 8/30
48000/48000 [==============================] - 1s 17us/step - loss: 0.1977 - accuracy: 0.9507
Epoch 9/30
48000/48000 [==============================] - 1s 17us/step - loss: 0.1959 - accuracy: 0.9529
Epoch 10/30
48000/48000 [==============================] - 1s 17us

48000/48000 [==============================] - 1s 22us/step - loss: 0.1674 - accuracy: 0.9630
Epoch 18/50
48000/48000 [==============================] - 1s 21us/step - loss: 0.1580 - accuracy: 0.9637
Epoch 19/50
48000/48000 [==============================] - 1s 21us/step - loss: 0.1740 - accuracy: 0.9632
Epoch 20/50
48000/48000 [==============================] - 1s 21us/step - loss: 0.1642 - accuracy: 0.9639
Epoch 21/50
48000/48000 [==============================] - 1s 21us/step - loss: 0.1675 - accuracy: 0.9645
Epoch 22/50
48000/48000 [==============================] - 1s 20us/step - loss: 0.1695 - accuracy: 0.9642
Epoch 23/50
48000/48000 [==============================] - 1s 20us/step - loss: 0.1648 - accuracy: 0.9652
Epoch 24/50
48000/48000 [==============================] - 1s 19us/step - loss: 0.1616 - accuracy: 0.9669
Epoch 25/50
48000/48000 [==============================] - 1s 20us/step - loss: 0.1612 - accuracy: 0.9655
Epoch 26/50
48000/48000 [==============================] -

48000/48000 [==============================] - 1s 15us/step - loss: 0.1904 - accuracy: 0.9651
Epoch 44/50
48000/48000 [==============================] - 1s 15us/step - loss: 0.1802 - accuracy: 0.9645
Epoch 45/50
48000/48000 [==============================] - 1s 15us/step - loss: 0.1829 - accuracy: 0.9658
Epoch 46/50
48000/48000 [==============================] - 1s 14us/step - loss: 0.1766 - accuracy: 0.9656
Epoch 47/50
48000/48000 [==============================] - 1s 15us/step - loss: 0.1807 - accuracy: 0.9653
Epoch 48/50
48000/48000 [==============================] - 1s 15us/step - loss: 0.1786 - accuracy: 0.9649
Epoch 49/50
48000/48000 [==============================] - 1s 15us/step - loss: 0.1859 - accuracy: 0.9644
Epoch 50/50
12000/12000 [==============================] - 0s 9us/step
Epoch 1/50
48000/48000 [==============================] - 1s 18us/step - loss: 1.1670 - accuracy: 0.7812
Epoch 2/50
48000/48000 [==============================] - 1s 16us/step - loss: 0.3539 - accura

...

48000/48000 [==============================] - 1s 14us/step - loss: 0.0283 - accuracy: 0.9910
Epoch 15/30
48000/48000 [==============================] - 1s 14us/step - loss: 0.0252 - accuracy: 0.9919
Epoch 16/30
48000/48000 [==============================] - 1s 14us/step - loss: 0.0213 - accuracy: 0.9932
Epoch 17/30
48000/48000 [==============================] - 1s 14us/step - loss: 0.0209 - accuracy: 0.9931
Epoch 18/30
48000/48000 [==============================] - 1s 14us/step - loss: 0.0183 - accuracy: 0.9939
Epoch 19/30
48000/48000 [==============================] - 1s 14us/step - loss: 0.0163 - accuracy: 0.9944
Epoch 20/30
48000/48000 [==============================] - 1s 14us/step - loss: 0.0158 - accuracy: 0.9949
Epoch 21/30
48000/48000 [==============================] - 1s 14us/step - loss: 0.0126 - accuracy: 0.9961
Epoch 22/30
48000/48000 [==============================] - 1s 14us/step - loss: 0.0129 - accuracy: 0.9955
Epoch 23/30
48000/48000 [==============================] -

48000/48000 [==============================] - 1s 13us/step - loss: 0.0447 - accuracy: 0.9859
Epoch 11/50
48000/48000 [==============================] - 1s 13us/step - loss: 0.0393 - accuracy: 0.9873
Epoch 12/50
48000/48000 [==============================] - 1s 13us/step - loss: 0.0344 - accuracy: 0.9891
Epoch 13/50
48000/48000 [==============================] - 1s 13us/step - loss: 0.0304 - accuracy: 0.9899
Epoch 14/50
48000/48000 [==============================] - 1s 13us/step - loss: 0.0269 - accuracy: 0.9912
Epoch 15/50
48000/48000 [==============================] - 1s 13us/step - loss: 0.0247 - accuracy: 0.9921
Epoch 16/50
48000/48000 [==============================] - 1s 13us/step - loss: 0.0228 - accuracy: 0.9927
Epoch 17/50
48000/48000 [==============================] - 1s 13us/step - loss: 0.0199 - accuracy: 0.9940
Epoch 18/50
48000/48000 [==============================] - 1s 13us/step - loss: 0.0183 - accuracy: 0.9944
Epoch 19/50
48000/48000 [==============================] -

48000/48000 [==============================] - 1s 11us/step - loss: 0.0072 - accuracy: 0.9978
Epoch 37/50
48000/48000 [==============================] - 1s 11us/step - loss: 0.0062 - accuracy: 0.9979
Epoch 38/50
48000/48000 [==============================] - 1s 11us/step - loss: 0.0054 - accuracy: 0.9984
Epoch 39/50
48000/48000 [==============================] - 1s 11us/step - loss: 0.0053 - accuracy: 0.9982
Epoch 40/50
48000/48000 [==============================] - 1s 11us/step - loss: 0.0068 - accuracy: 0.9978
Epoch 41/50
48000/48000 [==============================] - 1s 11us/step - loss: 0.0051 - accuracy: 0.9983
Epoch 42/50
48000/48000 [==============================] - 0s 10us/step - loss: 0.0056 - accuracy: 0.9981
Epoch 43/50
48000/48000 [==============================] - 0s 10us/step - loss: 0.0062 - accuracy: 0.9983
Epoch 44/50
48000/48000 [==============================] - 0s 10us/step - loss: 0.0050 - accuracy: 0.9985
Epoch 45/50
48000/48000 [==============================] -

48000/48000 [==============================] - 0s 10us/step - loss: 0.0347 - accuracy: 0.9890
Epoch 13/50
48000/48000 [==============================] - 0s 10us/step - loss: 0.0300 - accuracy: 0.9905
Epoch 14/50
48000/48000 [==============================] - 0s 10us/step - loss: 0.0273 - accuracy: 0.9916
Epoch 15/50
48000/48000 [==============================] - 0s 10us/step - loss: 0.0249 - accuracy: 0.9924
Epoch 16/50
48000/48000 [==============================] - 0s 10us/step - loss: 0.0217 - accuracy: 0.9930
Epoch 17/50
48000/48000 [==============================] - 0s 10us/step - loss: 0.0187 - accuracy: 0.9938
Epoch 18/50
48000/48000 [==============================] - 0s 10us/step - loss: 0.0181 - accuracy: 0.9941
Epoch 19/50
48000/48000 [==============================] - 1s 11us/step - loss: 0.0160 - accuracy: 0.9945
Epoch 20/50
48000/48000 [==============================] - 0s 10us/step - loss: 0.0150 - accuracy: 0.9949
Epoch 21/50
48000/48000 [==============================] -

48000/48000 [==============================] - 1s 21us/step - loss: 0.0763 - accuracy: 0.9780 - val_loss: 0.1580 - val_accuracy: 0.9599
Epoch 7/30
48000/48000 [==============================] - 1s 21us/step - loss: 0.0677 - accuracy: 0.9802 - val_loss: 0.1636 - val_accuracy: 0.9624
Epoch 8/30
48000/48000 [==============================] - 1s 21us/step - loss: 0.0570 - accuracy: 0.9829 - val_loss: 0.1757 - val_accuracy: 0.9628
Epoch 9/30
48000/48000 [==============================] - 1s 21us/step - loss: 0.0517 - accuracy: 0.9840 - val_loss: 0.1720 - val_accuracy: 0.9639
Epoch 10/30
48000/48000 [==============================] - 1s 21us/step - loss: 0.0447 - accuracy: 0.9870 - val_loss: 0.1802 - val_accuracy: 0.9631
Epoch 11/30
48000/48000 [==============================] - 1s 21us/step - loss: 0.0437 - accuracy: 0.9863 - val_loss: 0.2013 - val_accuracy: 0.9643
Epoch 12/30
48000/48000 [==============================] - 1s 21us/step - loss: 0.0377 - accuracy: 0.9890 - val_loss: 0.2140 - 